In [20]:
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error

In [27]:
database_path  = '/home/asma/Desktop/Diga Kala/Super cup/Sql/testset/testset.sqlite'

In [28]:
def creaet_sqlite_conn(path):
    try:
        conn = sqlite3.connect(path)
    except Error as e:
        print(e)
        
    conn.create_function("log2", 1, np.log2)
    
    return conn

In [29]:
con = creaet_sqlite_conn(database_path)

In [30]:
def run_query(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return pd.DataFrame(rows)

In [31]:
query_2 = """ SELECT query, (dcg/ idcg) as ndcg

FROM
(SELECT query,
       sum(click_count/log2(position+1)) as dcg,
	   sum(click_count/log2(right_position+1)) as idcg
FROM
	(SELECT *, row_number()over(ORDER by query, click_count DESC) as new_index
	from dk_table
	ORDER by query, click_count DESC) as sorted_clicks

join (SELECT position as right_position , row_number()over(ORDER by query, position) as new_index
	  FROM dk_table
	  ORDER by query, position) as sorted_positions
on sorted_clicks.new_index = sorted_positions.new_index

GROUP by query) """

In [33]:
x = run_query(con, query_2)

In [34]:
x

,0,1
0,405,0.804803
1,A10s,0.846427
2,A11,0.848403
3,A21,0.869239
4,A30,0.882900
...,...,...
995,یخچال اسنوا,0.879829
996,یخچال دوقلو,0.829705
997,یخچال ساید,0.857084
998,یخچال و فریزر,0.844439


In [53]:
query_1 = """SELECT query, sum(click_count/log2(position+1))/sum(click_count/log2(sorted_position+1)) as ndcg
FROM (
	SELECT query as sorted_position_query, position as sorted_position, click_count as sorted_position_clicks, ROW_NUMBER() OVER(ORDER BY query, position) AS id
	from  dk_table
	ORDER by query, position) as t1
JOIN (
	SELECT *, ROW_NUMBER() OVER(ORDER BY query, click_count DESC) AS id
	from dk_table
	ORDER by query,  click_count DESC) as t2

	on t2.id = t1.id
	group by query"""

In [54]:
x = run_query(conn, query_1)

In [55]:
x

,0,1
0,405,0.804803
1,A10s,0.846427
2,A11,0.848403
3,A21,0.869239
4,A30,0.882900
...,...,...
995,یخچال اسنوا,0.879829
996,یخچال دوقلو,0.829705
997,یخچال ساید,0.857084
998,یخچال و فریزر,0.844439


In [28]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[]


In [21]:
cursor = conn.cursor()
cursor.execute("select query, agg_array(click_count), agg_array(positions) from dk_table groupby query;")
rows = cursor.fetchall()
rows[0:5]

OperationalError: near "query": syntax error

In [7]:
data=cursor.execute('''SELECT * FROM dk_table''')
for column in data.description:
    print(column[0])

index
query
click_count
position


In [8]:
data = pd.DataFrame(rows, columns=['index', 'query', 'click_count', 'position'])

In [10]:
def compute_ndcg_for_dataset(dataset):
    df = dataset.groupby('query').agg({'click_count':lambda x: x.tolist(),'position':lambda x: x.tolist()})
    df['ndcg'] = df.apply(lambda x: compute_ndcg_per_query(x['click_count'],x['position']),axis=1)
    df = df.reset_index()
    return df[['query','ndcg']]

def compute_ndcg_per_query(clicks,position):
    if len(clicks)!= len(position):
        raise ValueError("clicks and positions len should be equal")
    dcg_list = np.array(clicks)/np.log2(np.array(position)+1)
    idcg_list = np.array(sorted(clicks,reverse=True))/np.log2(np.array(position)+1)
    dcg = np.sum(dcg_list)
    idcg = np.sum(idcg_list)
    return dcg/idcg

In [11]:
x = compute_ndcg_for_dataset(data)
x

,query,ndcg
0,405,0.804803
1,A10s,0.846427
2,A11,0.848403
3,A21,0.869239
4,A30,0.882900
...,...,...
995,یخچال اسنوا,0.879829
996,یخچال دوقلو,0.829705
997,یخچال ساید,0.857084
998,یخچال و فریزر,0.844439


In [17]:
query_1 = 'select query, agg_array(click_count), agg_array(positions) from dk_table groupby query;'